# This notebook demonstrates how to use kepler_kinematics to calculate stellar velocities

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import pymc3 as pm

import kepler_kinematics as kek

Calculating prior from 53395 stars


Let's load the Gaia-Kepler crossmatched catalog.

In [3]:
with fits.open("../../data/kepler_dr2_1arcsec.fits") as data:
    gaia = pd.DataFrame(data[1].data, dtype="float64")

In [4]:
gaia_rv = gaia.iloc[gaia.radial_velocity.values != 0.]

Select a star.

In [5]:
star_index = 0
df = gaia_rv.iloc[star_index]

First, let's calculate the 3D velocities of the star using its radial velocity.

In [6]:
# for i, row in enumerate(df):
xyz, vxyz = kek.simple_calc_vxyz(df["ra"], df["dec"],
                                 1./df["parallax"],
                                 df["pmra"], df["pmdec"],
                                 df["radial_velocity"])

vx, vy, vz = vxyz
x, y, z = xyz

Now let's infer the 3D velocities of the star. First we'll create arrays of position, parallax, and proper motion.

In [7]:
pos = [float(df["ra"]), float(df["dec"]), float(df["parallax"])]
pos_err = [float(df["ra_error"]), float(df["dec_error"]),
           float(df["parallax_error"])]
proper = [float(df["pmra"]), float(df["pmdec"])]
proper_err = [float(df["pmra_error"]), float(df["pmdec_error"])]

Get the prior that's built-in for Kepler stars.

In [8]:
mu, cov = kek.get_prior()

Run the velocity inference code.

In [ ]:
trace = kek.run_pymc3_model(pos, pos_err, proper, proper_err, mu, cov)

optimizing logp for variables: [vxyzD]


message: Optimization terminated successfully.
logp: -5535.2246067517335 -> -8.160599497356813
/Users/rangus/Applications/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vxyzD]


Pull out the results from the samples.

In [17]:
flat_samples = pm.trace_to_dataframe(trace)

params_inferred = np.median(flat_samples, axis=0)
upper = np.percentile(flat_samples, 84, axis=0)
lower = np.percentile(flat_samples, 16, axis=0)
errp = upper - params_inferred
errm = params_inferred - lower
std = np.std(flat_samples, axis=0)

results = pd.DataFrame(dict({"kepid": df["kepid"],
                             "vx": params_inferred[0], "vx_errp": errp[0], "vx_errm": errm[0], "vx_err": std[0],
                             "vy": params_inferred[1], "vy_errp": errp[1], "vy_errm": errm[1], "vy_err": std[1],
                             "vz": params_inferred[2], "vz_errp": errp[2], "vz_errm": errm[2], "vz_err": std[2],
                             "lndistance": params_inferred[3], "lndistance_errp": errp[3],
                             "lndistance_errm": errm[3], "lndistance_err": std[3]
                            }), index=[0])

In [39]:
print(f"{vx:.1f}, {float(results.vx.values):.1f} +/- {float(results.vx_err.values):.1f} km / s")
print(f"{vy:.1f}, {float(results.vy.values):.1f} +/- {float(results.vy_err.values):.1f} km / s")
print(f"{vz:.1f}, {float(results.vz.values):.1f} +/- {float(results.vz_err.values):.1f} km / s")

50.4 km / s, 53.4 +/- 7.2 km / s
215.2 km / s, 223.0 +/- 18.3 km / s
-0.3 km / s, 1.1 +/- 3.4 km / s
